# 『Pythonで学ぶ音声認識』（高島 遼一 2021）を Google Coalb で試す

まずデータの準備をおこなう。

Colab のデータはしばらく時間がたつと消えてしまうので，Google Drive に置いておく。

- Google Drive は無料だと 15 GB しか利用できない。
- もし容量が足りなければ，置いておくデータの選択が必要。`original` ディレクトリの中身は容量は大きいがダウンロードして展開するだけなので，Colab を起動するたびにダウンロードして展開する方針でもいいのかもしれない。

# 第3章 音声処理の基礎と特徴量抽出

## 第3.1節 データの準備

### リポジトリを複製（git clone）する

- 本家から fork したリポジトリを利用する

In [1]:
!git clone https://github.com/dfukagaw28/python_asr.git

Cloning into 'python_asr'...
remote: Enumerating objects: 132, done.
remote: Counting objects: 100% (128/128), done.
remote: Compressing objects: 100% (72/72), done.
remote: Total 132 (delta 53), reused 117 (delta 46), pack-reused 4
Receiving objects: 100% (132/132), 213.61 KiB | 7.12 MiB/s, done.
Resolving deltas: 100% (53/53), done.


In [2]:
%mkdir -p /content/python_asr/data/original
%cd /content/python_asr/data/original

/content/python_asr/data/original


In [3]:
import hashlib
from pathlib import Path
from urllib.parse import urlparse

import requests
from tqdm import tqdm

def download(file_url, dst_path, hash_md5=None):
    if not Path(dst_path).exists():
        file_size = int(requests.head(file_url).headers["content-length"])
        res = requests.get(file_url, stream=True)
        pbar = tqdm(total=file_size, unit="B", unit_scale=True)
        with open(dst_path, 'wb') as file:
            for chunk in res.iter_content(chunk_size=1024):
                file.write(chunk)
                pbar.update(len(chunk))
            pbar.close()
    if hash_md5:
        with open(dst_path, 'rb') as f:
            file_md5 = hashlib.md5(f.read()).hexdigest()
            assert file_md5 == hash_md5


In [4]:
# 音声ファイル(jsutコーパス. zip形式)をダウンロード
download(
    'https://drive.google.com/u/0/uc?id=1f7bIQfwWdFOxeaYzs5Cw-HTcA8uwQ8qp&export=download&confirm=t',
    'jsut_ver1.1.zip',
    'f3e9dcd3e230880db494f9853b0517fb',
)

2.69GB [01:04, 42.0MB/s]


In [5]:
# jsutコーパスのラベルデータをダウンロード
download(
    'https://github.com/sarulab-speech/jsut-label/archive/1978271.zip',
    '1978271.zip',
    '4958cd831b28769ad72ab40666f595e3',
)

9.40MB [00:01, 6.47MB/s]


In [6]:
# zip ファイルを展開する
import shutil
import zipfile
from tqdm import tqdm

for filename in ['jsut_ver1.1.zip', '1978271.zip']:
    with zipfile.ZipFile(filename) as zf:
        for member in tqdm(zf.infolist(), desc='Extracting '):
            if not Path(member.filename).exists():
                zf.extract(member)

Path('jsut-label-1978271ca6212e1ea742da8f149160f5679e8971').rename('jsut-label-master')  

Extracting : 100%|██████████| 5015/5015 [00:01<00:00, 4712.36it/s]


### ダウンサンプリング処理（00prepare/01prepare_wav.py）

JSUT コーパスの音声データを 48kHz から 16kHz に変換する。

pysox パッケージを用いる。
- https://github.com/rabitt/pysox
- http://sox.sourceforge.net/

In [7]:
%cd /content/python_asr/00prepare

/content/python_asr/00prepare


In [8]:
!apt-get install -qy libsox-fmt-all sox

Reading package lists...
Building dependency tree...
Reading state information...
The following package was automatically installed and is no longer required:
  libnvidia-common-460
Use 'apt autoremove' to remove it.
The following additional packages will be installed:
  libao-common libao4 libid3tag0 libmad0 libmagic-mgc libmagic1
  libopencore-amrnb0 libopencore-amrwb0 libsox-fmt-alsa libsox-fmt-ao
  libsox-fmt-base libsox-fmt-mp3 libsox-fmt-oss libsox-fmt-pulse libsox3
Suggested packages:
  libaudio2 file
The following NEW packages will be installed:
  libao-common libao4 libid3tag0 libmad0 libmagic-mgc libmagic1
  libopencore-amrnb0 libopencore-amrwb0 libsox-fmt-all libsox-fmt-alsa
  libsox-fmt-ao libsox-fmt-base libsox-fmt-mp3 libsox-fmt-oss libsox-fmt-pulse
  libsox3 sox
0 upgraded, 17 newly installed, 0 to remove and 45 not upgraded.
Need to get 942 kB of archives.
After this operation, 7,441 kB of additional disk space will be used.
Get:1 http://archive.ubuntu.com/ubuntu bionic

In [9]:
%pip install sox

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [20]:
%run 01prepare_wav.py

../data


100%|██████████| 5000/5000 [01:27<00:00, 57.05it/s]


In [21]:
!ls ../data/wav | wc
!du -sh ../data/wav

   5000    5000   95000
755M	../data/wav


In [22]:
!head ../data/label/all/wav.scp

BASIC5000_0001 /content/python_asr/data/wav/BASIC5000_0001.wav
BASIC5000_0002 /content/python_asr/data/wav/BASIC5000_0002.wav
BASIC5000_0003 /content/python_asr/data/wav/BASIC5000_0003.wav
BASIC5000_0004 /content/python_asr/data/wav/BASIC5000_0004.wav
BASIC5000_0005 /content/python_asr/data/wav/BASIC5000_0005.wav
BASIC5000_0006 /content/python_asr/data/wav/BASIC5000_0006.wav
BASIC5000_0007 /content/python_asr/data/wav/BASIC5000_0007.wav
BASIC5000_0008 /content/python_asr/data/wav/BASIC5000_0008.wav
BASIC5000_0009 /content/python_asr/data/wav/BASIC5000_0009.wav
BASIC5000_0010 /content/python_asr/data/wav/BASIC5000_0010.wav


### 3種類のテキストラベルを抽出（00prepare/02prepare_label.py）

In [23]:
!head ../data/original/jsut-label-master/text_kana/basic5000.yaml

BASIC5000_0001:
  text_level0: 水をマレーシアから買わなくてはならないのです。
  kana_level0: みずをまれーしあからかわなくてわならないのです
  text_level1: 水をマレーシアから買わなくてはならないのです。
  text_level2: 水をマレーシアから買わなくてはならないのです。
  kana_level2: みずをまれーしあからかわなくてわならないのです
  kana_level3: みずをまれーしあからかわなくてわならないのです
  phone_level3: m-i-z-u-o-m-a-r-e-e-sh-i-a-k-a-r-a-k-a-w-a-n-a-k-u-t-e-w-a-n-a-r-a-n-a-i-n-o-d-e-s-u
BASIC5000_0002:
  text_level0: 木曜日、停戦会談は、何の進展もないまま終了しました。


In [24]:
%run 02prepare_label.py

In [25]:
!wc -l ../data/label/all/text_*

   5000 ../data/label/all/text_char
   5000 ../data/label/all/text_kana
   5000 ../data/label/all/text_phone
  15000 total


In [26]:
!head ../data/label/all/text_*

==> ../data/label/all/text_char <==
BASIC5000_0001 水 を マ レ ー シ ア か ら 買 わ な く て は な ら な い の で す
BASIC5000_0002 木 曜 日 停 戦 会 談 は 何 の 進 展 も な い ま ま 終 了 し ま し た
BASIC5000_0003 上 院 議 員 は 私 が デ ー タ を ゆ が め た と 告 発 し た
BASIC5000_0004 １ 週 間 し て そ の ニ ュ ー ス は 本 当 に な っ た
BASIC5000_0005 血 圧 は 健 康 の パ ロ メ ー タ ー と し て 重 要 で あ る
BASIC5000_0006 週 に 四 回 フ ラ ン ス の 授 業 が あ り ま す
BASIC5000_0007 許 可 書 が な け れ ば こ こ へ は 入 れ な い
BASIC5000_0008 大 声 で 泣 き な が ら 女 の 子 は 母 親 を 探 し て い た
BASIC5000_0009 無 罪 の 人 々 は も ち ろ ん 放 免 さ れ た
BASIC5000_0010 末 期 試 験 に 備 え て 本 当 に 気 合 い を 入 れ て 勉 強 し な き ゃ

==> ../data/label/all/text_kana <==
BASIC5000_0001 み ず を ま れ ー し あ か ら か わ な く て わ な ら な い の で す
BASIC5000_0002 も く よ ー び て ー せ ん か い だ ん わ な ん の し ん て ん も な い ま ま し ゅ ー り ょ ー し ま し た
BASIC5000_0003 じ ょ ー い ん ぎ ー ん わ わ た し が で ー た を ゆ が め た と こ く は つ し た
BASIC5000_0004 い っ し ゅ ー か ん し て そ の に ゅ ー す わ ほ ん と ー に な っ た
BASIC5000_0005 け つ あ つ わ け ん こ ー の ぱ ろ め ー た ー と し て じ ゅ ー よ ー で あ る
BASIC5000_0006 し ゅ ー に よ ん か い ふ ら ん す

### 学習/開発/評価用のデータセットに分割（00prepare/03subset_data.py）

```
all          全データ
train_large  学習データ（大）
train_small  学習データ（小）
dev          開発データ
test         評価データ
```

In [27]:
%run 03subset_data.py

In [28]:
!for f in all train_large train_small dev test; do echo; wc -l ../data/label/$f/*; done


   5000 ../data/label/all/text_char
   5000 ../data/label/all/text_kana
   5000 ../data/label/all/text_phone
   5000 ../data/label/all/wav.scp
  20000 total

   4500 ../data/label/train_large/text_char
   4500 ../data/label/train_large/text_kana
   4500 ../data/label/train_large/text_phone
   4500 ../data/label/train_large/wav.scp
  18000 total

  1000 ../data/label/train_small/text_char
  1000 ../data/label/train_small/text_kana
  1000 ../data/label/train_small/text_phone
  1000 ../data/label/train_small/wav.scp
  4000 total

   250 ../data/label/dev/text_char
   250 ../data/label/dev/text_kana
   250 ../data/label/dev/text_phone
   250 ../data/label/dev/wav.scp
  1000 total

   250 ../data/label/test/text_char
   250 ../data/label/test/text_kana
   250 ../data/label/test/text_phone
   250 ../data/label/test/wav.scp
  1000 total


## 生成したデータを Google Drive に保存する

In [29]:
%cd /content/python_asr/data

/content/python_asr/data


In [31]:
!du -sm *

6	label
6039	original
755	wav


In [32]:
!du -sm original/*

9	original/1978271.zip
65	original/jsut-label-master
3402	original/jsut_ver1.1
2565	original/jsut_ver1.1.zip


In [ ]:
!rm original/*.zip

In [33]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [38]:
!mkdir -p /content/drive/MyDrive/data/python_asr

In [39]:
!mv /content/python_asr/data/* /content/drive/MyDrive/data/python_asr/